In [ ]:
import pandas as pd
from langchain_community.document_loaders import CSVLoader, PyPDFLoader
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Milvus
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_gigachat import GigaChat
from langchain_community.retrievers import BM25Retriever
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from langchain.retrievers import EnsembleRetriever

def create_milvus_langchain_agent(agent_name: str, system_prompt: str, file_paths: list):
    # Загрузка и обработка данных из файлов
    documents = []
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)  # Разбиение текста на чанки

    for file_path in file_paths:
        if file_path.endswith('.pdf'):
            loader = PyPDFLoader(file_path)
            pdf_docs = loader.load()
            split_docs = text_splitter.split_documents(pdf_docs)
            documents.extend(split_docs)
        else:
            raise ValueError("Неподдерживаемый формат файла. Поддерживаются только CSV, Excel и PDF файлы.")

    # Инициализация модели для эмбеддингов
    embeddings = HuggingFaceEmbeddings(model_name='deepvk/USER-bge-m3', cache_folder='../cache/embedding/')

    # Создание векторной базы данных Milvus
    vector_db = Milvus.from_documents(
        documents,
        embeddings,
        connection_args={"host": "localhost", "port": "19530"},
        collection_name=agent_name
    )

    # Создание BM25Retriever
    bm25_retriever = BM25Retriever.from_documents(documents, k=3)  # k — количество возвращаемых документов

    # Создание EnsembleRetriever
    ensemble_retriever = EnsembleRetriever(
        retrievers=[vector_db.as_retriever(), bm25_retriever],
        weights=[0.7, 0.3]  # Веса для каждого ретривера (70% для Milvus, 30% для BM25)
    )

    # Создание цепочки Langchain
    template = system_prompt + "\n\nКонтекст:\n{context}\n\nВопрос: {question}\n\nОтвет:"
    prompt = ChatPromptTemplate.from_template(template)

    # Инициализация GigaChat LLM
    llm = GigaChat(credentials="your_gigachat_credentials_here")

    # Создание цепочки
    chain = (
        {"context": ensemble_retriever, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )

    return chain

# Пример использования функции
agent_name = "RUTUBE_Assistant"
system_prompt = "Ты интеллектуальный помощник компании RUTUBE. Отвечай на вопросы, используя предоставленный контекст."
file_paths = ["../data/01_База_знаний.xlsx", "../data/02_Реальные_кейсы.xlsx", "../data/some_document.pdf"]

chain = create_milvus_langchain_agent(agent_name, system_prompt, file_paths)

# Тестирование цепочки
response = chain.invoke("Как подключить монетизацию на RUTUBE?")
print(response)

In [ ]:
from langchain_community.vectorstores import Milvus
from langchain_community.retrievers import BM25Retriever
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_gigachat import GigaChat
from langchain.retrievers import EnsembleRetriever

def create_chain_from_existing_collection(system_prompt: str, collection_name: str, bm25_documents: list):
    """
    Создает цепочку (chain) на основе существующей коллекции Milvus и BM25Retriever.

    :param system_prompt: Системный промпт для LLM.
    :param collection_name: Имя существующей коллекции Milvus.
    :param bm25_documents: Документы для BM25Retriever (список объектов Document).
    :return: Цепочка (chain) для генерации ответов.
    """
    # Инициализация модели для эмбеддингов
    embeddings = HuggingFaceEmbeddings(model_name='deepvk/USER-bge-m3', cache_folder='../cache/embedding/')

    # Подключение к существующей коллекции Milvus
    vector_db = Milvus(
        embedding_function=embeddings,
        collection_name=collection_name,
        connection_args={"host": "localhost", "port": "19530"}
    )

    # Создание BM25Retriever на основе предоставленных документов
    bm25_retriever = BM25Retriever.from_documents(bm25_documents, k=3)  # k — количество возвращаемых документов

    # Создание EnsembleRetriever
    ensemble_retriever = EnsembleRetriever(
        retrievers=[vector_db.as_retriever(), bm25_retriever],
        weights=[0.7, 0.3]  # Веса для каждого ретривера (70% для Milvus, 30% для BM25)
    )

    # Создание цепочки Langchain
    template = system_prompt + "\n\nКонтекст:\n{context}\n\nВопрос: {question}\n\nОтвет:"
    prompt = ChatPromptTemplate.from_template(template)

    # Инициализация GigaChat LLM
    llm = GigaChat(credentials="your_gigachat_credentials_here")

    # Создание цепочки
    chain = (
        {"context": ensemble_retriever, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )

    return chain

# Пример использования функции
system_prompt = "Ты интеллектуальный помощник компании RUTUBE. Отвечай на вопросы, используя предоставленный контекст."
collection_name = "RUTUBE_Assistant"  # Имя существующей коллекции Milvus

# Предположим, что у нас уже есть документы для BM25Retriever
bm25_documents = [
    Document(page_content="Вопрос: Как подключить монетизацию?\nОтвет: Для подключения монетизации нужно...", metadata={"source": "data.csv"}),
    Document(page_content="Вопрос: Что нельзя публиковать на RUTUBE?\nОтвет: Чужой контент без разрешения...", metadata={"source": "data.csv"}),
]

# Создание цепочки
chain = create_chain_from_existing_collection(system_prompt, collection_name, bm25_documents)

# Тестирование цепочки
response = chain.invoke("Как подключить монетизацию на RUTUBE?")
print(response)